# Decision Tree From Scratch (ZWM - 03/21/17)

###### Imports (All imports here for cleanliness)

In [389]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import math

###### Silly function to generate the test data, just to keep it clean

In [390]:
def get_data():
    """
    Get some data, either pretend user data or the Iris dataset
    ------
    """
    # An oversimplified dataset for easy visualization while learning
    """headers = ['referral','country','faq','age','service']
    my_data=[['slashdot','USA','yes',18,'None'],
        ['google','France','yes',17,'Premium'],
        ['google','France','yes',20,'Premium'],
        ['reddit','USA','yes',24,'Basic'],
        ['kiwitobes','France','yes',23,'Basic'],
        ['google','UK','no',21,'Premium'],
        ['(direct)','New Zealand','no',12,'None'],
        ['(direct)','UK','no',21,'Basic'],
        ['google','USA','no',24,'Premium'],
        ['slashdot','France','yes',19,'None'],
        ['slashdot','UK','yes',31,'Basic'],
        ['reddit','USA','no',18,'None'],
        ['google','UK','no',18,'None'],
        ['kiwitobes','UK','no',19,'None'],
        ['reddit','New Zealand','yes',12,'Basic'],
        ['slashdot','UK','no',21,'None'],
        ['google','UK','yes',18,'Basic'],
        ['kiwitobes','France','yes',19,'Basic']]
        """
    # The Iris data set from SKLearn 
    # (http://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html)
    from sklearn.datasets import load_iris
    iris = load_iris()
    my_data = []
    for x,y in zip(iris.data.tolist(),iris.target.tolist()):
        my_data.append(x)
        my_data[len(my_data)-1].append(y)
    return my_data

###### Tree building helper functions (the real mechanics between node finding)

In [391]:
def split_data(data,colnum,value):
    """
    Returns: Two sets of data from the initial data. Set 1 contains those that passed
    the condition of data[colnum] >= value
    ----------
    Input: The dataset, the column to split on, the value on which to split
    """
    splitter = None
    if isinstance(value, int) or isinstance(value,float):
        splitter = lambda x: x[colnum] >= value
    else:
        splitter = lambda x: x[colnum] == value
    set1 = [row for row in data if splitter(row)]
    set2 = [row for row in data if not splitter(row)]
    return set1,set2

def count_target_values(data):
    """
    Returns: A dictionary of target variable counts in the data
    """
    results = {}
    for row in data:
        if row[-1] not in results:
            results[row[-1]] = 0
        results[row[-1]] += 1
    return results
    
def entropy(data):
    """
    Returns: Entropy of the data set, based on target values. 
    ent = Sum(-p_i Log(p_i), i in unique targets) where p is the percentage of the
    data with the ith label.
    Sidenote: We're using entropy as our measure of good splits. It corresponds to 
    information gained by making this split. If the split results in only one target type
    then the entropy new sets entropy is 0. If it results in a ton of different targets, the
    entropy will be high. 
    """
    results = count_target_values(data)
    log2=lambda x:math.log(x)/math.log(2)
    ent=0.
    for r in results.keys():
        p=float(results[r])/len(data) 
        ent-=p*log2(p)
    return ent  

###### The actual tree maker with node class for nesting (also has prediction and tree printer functions)

In [392]:
class tree_node:
    def __init__(self,col=-1,value=None,results=None,label=None,tb=None,fb=None):
        self.col=col # column index of criteria being tested
        self.value=value # vlaue necessary to get a true result
        self.results=results # dict of results for a branch, None for everything except endpoints
        self.tb=tb # true decision nodes 
        self.fb=fb # false decision nodes

def train(data, scorefun=entropy, use_features=None):
    if len(data) == 0: return tree_node()
    current_score = scorefun(data)

    best_gain = 0.0
    best_criteria = None
    best_sets = None
    
    if use_features==None:
        use_features = [i for i in range(len(data[0]) - 1)]
    for col in use_features:
        # find different values in this column
        column_values = set([row[col] for row in data])

        # for each possible value, try to divide on that value
        for value in column_values:
            set1, set2 = split_data(data, col, value)

            # Information gain
            p = float(len(set1)) / len(data)
            gain = current_score - p*scorefun(set1) - (1-p)*scorefun(set2)
            if gain > best_gain and len(set1) > 0 and len(set2) > 0:
                best_gain = gain
                best_criteria = (col, value)
                best_sets = (set1, set2)

    if best_gain > 0:
        trueBranch = train(best_sets[0],use_features=use_features)
        falseBranch = train(best_sets[1],use_features=use_features)
        return tree_node(col=best_criteria[0], value=best_criteria[1],
                tb=trueBranch, fb=falseBranch)
    else:
        return tree_node(results=count_target_values(data))

def print_tree(tree,indent=''):
    if tree.results!=None: # if this is a end node
        print str(tree.results)
    else:
        print 'Column ' + str(tree.col)+' : '+str(tree.value)+'? '
        # Print the branches
        print indent+' True: ',
        print_tree(tree.tb,indent+indent)
        print indent+' False: ',
        print_tree(tree.fb,indent+indent)

def predict(newdata, tree):
    if tree.results!=None: # if this is a end node
        return tree.results.keys()[0]
    
    if isinstance(newdata[tree.col], int) or isinstance(newdata[tree.col],float):
        if newdata[tree.col] >= tree.value:
            return predict(newdata, tree.tb)
            
        else:
            return predict(newdata, tree.fb)
    else:
        if newdata[tree.col] == tree.value:
            return predict(newdata, tree.tb)
        else:
            return predict(newdata, tree.fb) 
        
def scorer(results):
    correct = 0
    for i in results:
        if i[0] == i[1]:
            correct+=1
    return float(correct)/float(len(results))

### Actually get the data, train, test, and report accuracy (with bagging to see accuracy of our model)

In [393]:
data = get_data()

"""
--- Set Options ---
random_forest chooses whether to use Random Forest Classifier mode.
Random Forest Classifier has bagging and multiple trees generated
with randomly selected features (sqrt(numFeatures) per tree). 

Not random_forest creates a single decision tree with/without bagging
depending on if bagging == True.
"""
random_forest = True
bagging = True
num_trees = 100
bagging_value = 100
random.seed(1234)
np.random.seed(1234)

In [394]:
#Shuffle and split the data (shuffle because it comes sequentially by class)
random.shuffle(data)
train_split_index = int(0.7*len(data))
train_data = data[0:train_split_index]
test_data = data[train_split_index+1:]
accuracy_values = []
result_dict = {}

if not random_forest:
    num_trees = 1
if not bagging:
    bagging_value = 1
    
for _ in range(num_trees):
    valid_cols = []
    if random_forest:
        num_cols_select = int(math.sqrt(len(data[0]))+0.5)
        possible_cols = [i for i in range(len(data[0]) - 1)]
        for _ in range(num_cols_select):
            random.shuffle(possible_cols)
            valid_cols.append(possible_cols.pop())
    else:
        valid_cols = [i for i in range(len(data[0]) - 1)]
    
    bagged_data = [random.choice(train_data) for _ in range(bagging_value)]
    tree = train(bagged_data,use_features=valid_cols)
    # print_tree(tree,'---') # For visualizing the tree splits

    results = []
    for i,new_data in enumerate(test_data):
        if i not in result_dict.keys(): 
            result_dict[i] = []
        ypred = predict(new_data, tree)
        y = new_data[-1]
        results.append([y,ypred])
        result_dict[i].append(ypred)
    accuracy_values.append(scorer(results))

from collections import Counter
bag_result_mode = []
for i,data in enumerate(test_data):
    bag_result_mode.append([Counter(result_dict[i]).most_common(1)[0][0],test_data[i][-1]])
    
print "Accuracy for Bagged Result (Mode): " + str(scorer(bag_result_mode))  
print "Average Accuracy for Single Tree: " + str(sum(accuracy_values)/len(accuracy_values))

Accuracy for Bagged Result (Mode): 0.931818181818
Average Accuracy for Single Tree: 0.8925


### Compare with SkLearn Classifiers ("professional" trees)

In [395]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
X, Y = load_iris().data, load_iris().target
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.3)

sktree = DecisionTreeClassifier(criterion='entropy')
skrf = RandomForestClassifier(n_estimators = 100,criterion='entropy')

sktree.fit(x_train,y_train)
skrf.fit(x_train,y_train)

print "Accuracy of Decision Tree from SkLearn: " + str(sktree.score(x_test,y_test))
print "Accuracy of Random Forest from SkLearn: " + str(skrf.score(x_test,y_test))

Accuracy of Decision Tree from SkLearn: 0.977777777778
Accuracy of Random Forest from SkLearn: 0.955555555556
